In [1]:
# 1. 导入必要的库
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import pandas as pd
import os

In [2]:
# 获取当前脚本所在的目录并拼接路径
# csv_file_path = os.path.join(os.path.dirname(__file__), "smile_ir.csv")
# print(csv_file_path)

In [3]:
def calculate_top10_similar(smiles_input, csv_file_path, output_csv_path):
    """
    输入一个 SMILES 字符串，将其转换为分子对象并生成指纹，
    然后计算与给定 CSV 文件中第一列（列名为 "smiles"）的所有分子的相似性，
    返回相似度最高的前10个，并将匹配到的整行数据（插入 similarity 列）保存到新的 CSV 文件中。

    参数：
    ----------
    smiles_input : str
        输入的目标分子 SMILES 字符串。

    csv_file_path : str
        包含 SMILES 数据的 CSV 文件路径，要求其列名中至少有一个 "smiles" 列。

    output_csv_path : str
        输出包含前 10 个最相似分子的 CSV 文件路径。

    返回值：
    ----------
    str
        返回 "成功" 表示处理完成。

    示例：
    ----------
    smiles_input = "CCO"
    csv_file_path = "smile_ir.csv"
    output_csv_path = "top10_similar.csv"
    result = calculate_top10_similar(smiles_input, csv_file_path, output_csv_path)
    print(result)
    """

    # 1. 读取 CSV，让 Pandas 将第一行为列名，从第二行起作为数据
    data = pd.read_csv(csv_file_path)  # 相当于 header=0
    if data.empty:
        raise ValueError(f"CSV 文件无效或为空: {csv_file_path}")

    # 确保存在名为 "smiles" 的列
    if "smiles" not in data.columns:
        raise ValueError(f"CSV 文件中未找到名为 'smiles' 的列: {csv_file_path}")

    # 2. 获取 SMILES 列的所有分子
    smiles_list = data["smiles"].dropna().tolist()

    # 3. 将输入 SMILES 转换为分子对象
    target_mol = Chem.MolFromSmiles(smiles_input)
    if target_mol is None:
        raise ValueError(f"输入的 SMILES 字符串无效: {smiles_input}")

    # 4. 生成目标分子的 Morgan 指纹
    target_fp = AllChem.GetMorganFingerprintAsBitVect(target_mol, radius=2)

    # 5. 计算相似度
    similarity_results = []
    for s in smiles_list:
        mol = Chem.MolFromSmiles(s)
        if mol is not None:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2)
            similarity = DataStructs.TanimotoSimilarity(target_fp, fp)
            similarity_results.append((s, similarity))
        else:
            print(f"无效的 SMILES 字符串：{s}")

    # 6. 相似度降序排序，取前 10
    similarity_results.sort(key=lambda x: x[1], reverse=True)
    top10 = similarity_results[:10]

    # 7. 准备一个新的 DataFrame 用于存放结果
    #    在原有列的第 1 列位置插入 "similarity"
    new_columns = list(data.columns)
    # 如果 "similarity" 已经存在，可先根据需求去重处理
    if "similarity" not in new_columns:
        new_columns.insert(1, "similarity")
    top10_df = pd.DataFrame(columns=new_columns)

    # 8. 将 top10 的分子行复制到新的 DataFrame 中，并插入 similarity
    for smiles_val, sim in top10:
        # 在 data 中查找与该 smiles 匹配的行
        matched_rows = data.loc[data["smiles"] == smiles_val].copy()
        if not matched_rows.empty:
            # 在第 1 列插入 similarity 列
            matched_rows.insert(1, "similarity", sim)
            # 与 top10_df 的列顺序保持一致
            matched_rows = matched_rows[top10_df.columns]
            # 拼接
            top10_df = pd.concat([top10_df, matched_rows], ignore_index=True)

    # 9. 将结果保存到新的 CSV 文件（只会有一行表头）
    top10_df.to_csv(output_csv_path, index=False)

    return "成功"

In [4]:
input_smiles = "Oc1ccc(Oc2ccc(OCc3ccc(O)cc3)cc2)cc1"
calculate_top10_similar(input_smiles, "smile_ir.csv","top10_similar.csv")

'成功'